# **Recitation 6**

In [ ]:
# Click the "Play" button on the left to import packages.
%pip install -i https://pypi.gurobi.com gurobipy;
import gurobipy
import cvxpy as cp
import numpy as np
import pandas as pd
 
# Below function will simplify the conversion of numpy arrays into dataframes
def to_df(a): # turns array into pandas dataframe
  df = pd.DataFrame(data    = a[1:,1:] , # values
                    index   = a[1:,0]  , # 1st column as rows
                    columns = a[0,1:]  ) # 1st row as the column names
  df = df.astype('float') # Turn dataframe entries into floats
  return df

Looking in indexes: https://pypi.gurobi.com
     |████████████████████████████████| 11.1 MB 17.7 MB/s 


**Reference Code**

Recall the following transportation problem from Lecture 6 ([link](https://canvas.cornell.edu/courses/29568/files/4270031?wrap=1)).

In [ ]:
# Define supply and demand dictionaries
supply = {'Seattle'   : 350,
          'San Diego' : 600}

demand = {'New York'  : 325,
          'Chicago'   : 300, 
          'Topeka'    : 275,
          'Unused'    : 50}

# Define the cost of transportation
cost = to_df(np.array([[ '',          'New York',   'Chicago',  'Topeka', 'Unused'],
                       ['Seattle' ,   225,          153,        162,      0],
                       ['San Diego',  225,          162,        126,      0]]))

In [ ]:
# Define function to solve transportation problem
def transport(supply, demand, cost):

  # Enforce all non-negative supply and demand
  assert all((val >= 0) for val in supply.values())
  assert all((val >= 0) for val in demand.values())

  # Check supply and demand match
  assert sum(supply.values()) == sum(demand.values())

  # Check there is cost matches supply-demand
  assert set(cost.index) == set(supply.keys())
  assert set(cost.columns) == set(demand.keys())
  
  # Check all costs are numeric and nonnegative
  assert np.all(cost.to_numpy() >= 0)
  
  # Create transportation variables
  trans = {}
  for orig in supply.keys():
    for dest in demand.keys():
      trans[orig, dest] = cp.Variable(nonneg = True) # Units to be shipped

  # Define objective
  total_cost = sum(cost.loc[orig,dest] * trans[orig,dest] for orig in supply.keys() for dest in demand.keys())

  # Create constraints
  constraints = []
  for orig in supply.keys():
    constraints += [ sum(trans[orig,dest] for dest in demand.keys()) == supply[orig] ]
  for dest in demand.keys():
    constraints += [ sum(trans[orig,dest] for orig in supply.keys()) == demand[dest]  ]

  # Solve problem
  prob = cp.Problem(cp.Minimize(total_cost), constraints)
  prob.solve(solver = cp.GUROBI)

  # Print objective
  print('objective = %s' % prob.value)

  # Print transportation
  print('trans = ')
  output = pd.DataFrame(index=cost.index, columns=cost.columns, dtype=float)
  for orig in supply.keys():
    for dest in demand.keys():
      output.loc[orig, dest] = trans[orig,dest].value
  print(output)

In [ ]:
# Solve transportation problem 
transport(supply, demand, cost)

Restricted license - for non-production use only - expires 2022-01-13
objective = 153675.0
trans = 
           New York  Chicago  Topeka  Unused
Seattle         0.0    300.0     0.0    50.0
San Diego     325.0      0.0   275.0     0.0


**Exercise**

A small manufacturing operation produces six kinds of parts, using three machines. For the coming month, a certain number of each part is needed (this is shown in the "required" row below). There is also an upper limit to the number of parts that can be made on each machine (these are given in the "capacity" column below). To complicate matters, it does not cost the same amount to make the same part on different machines (the cost is shown in the cells of the table below).

The relevant information can be summarized as follows:

**Table of COSTS (per part), CAPACITY (number of parts per machine), and REQUIRED Number of Parts (per part type):**

\begin{array}{cccccccc}
&  & &\textbf{Part} & & & &\\
\textbf{Machine}& 1& 2& 3& 4& 5& 6 & \textbf{Parts Capacity}\\
1 & 3& 3& 2& 5& 2& 1 & 80\\
2 & 4& 1& 1& 2& 2& 1 & 30\\
3 & 2& 2& 5& 1& 1& 2 & 160\\
\textbf{Parts Required} & 10 & 40& 60& 20& 20& 30 &\\
\end{array}


**(a)** Map this problem to the transportation model: Treat the machines as the origins, and the parts as the destinations. Using this, create data for *supply* (Python dictionary), *demand* (Python dictionary), and *cost* (Python/Pandas DataFrame) to be passed to the `transport` function to solve the problem. How many of each part should be produced on each machine, so as to minimize total cost?

Hint: You will need to create a new "product" to represent the unused capacity on the machines.

In [ ]:
### CREATE DATA FOR PART (a) BELOW ###


**(b)** If the capacity of machine 2 is increased to 50, the manufacturer may be able to reduce the total cost of production somewhat. How much is the total cost reduced, and in what way does the production plan change?

Hint: Define new data that corresponds with this new senario. Pass the data to the same function you used for part (a).

In [ ]:
## CREATE DATA FOR PART (b) BELOW ##


**(c)** Now suppose that the capacity limit is not by number of parts produced by each machine---as had been the case in (a)---but rather by number of hours each machine is in-use. The maximum number of hours per machine is given in the "Time-Capacity" column below. Also, suppose it takes a different number of hours to make the same part on different machines (given in cells of the table below). The information is summarized as follows:

**Table of TIMES (per part) and Maximum Time CAPACITY (per machine)**

\begin{array}{cccccccc}
&  & &\textbf{Part} & & & &\\
\textbf{Machine}& 1& 2& 3& 4& 5& 6 & \textbf{Time-Capacity}\\
1 & 1.3& 1.3& 1.2& 1.5& 1.2& 1.1 & 50\\
2 & 1.4& 1.1& 1.1& 1.2& 1.2& 1.1 & 90\\
3 & 1.2& 1.2& 1.5& 1.1& 1.1& 1.2 & 175\\
\end{array}
The data for the problem is given to you in the below code-cell. Do the following:
* Using the reference code as a starting template, define a new function called `manufacture` that takes in the data and solves this modified problem with time constraints. 

  You are *still minimizing cost*---so the cost table from part (a) still applies. Thus, you must now incorporate an *additional* table where you must consider the time constraints as well. Moreover, the constraint on the *required* number of parts from part (a) *also still applies*. 
  
  It is only that the capacity constraint on the *number* of parts from part (a) that nolonger applies: it has been replaced by the new time-capacity constraint.
* Thus, compared to the `transport`, you must modify (among other things) the supply constraint so that it limits total time of production at each machine.
* Additionally, after the optimization is complete, calculate the amount of time used by each machine. Print the *extra time* for each machine i.e. the number of hours each machine is below capacity.
* How is the new optimal solution different? On which machines is all available time used?

Hint: Notice that the data given to you does not have a "unused product" like we told you to create in part (a). This is because the "unused" category is a "dummy" field created to comply with the transportation problem's specific requirement that supply must *equal* demand. In the current problem, the time used need only be *less* than capacity. Incorporate this into the new function you write as well: This aspect only requires changing one equality constraint in the template into an inequality.

In [ ]:
capacity_c = {'M1' : 50,
            'M2' : 90,
            'M3' : 175}

required_c = {'P1' : 10,
            'P2' : 40,
            'P3' : 60,
            'P4' : 20,
            'P5' : 20,
            'P6' : 30}

cost_c = to_df(np.array([[ '' ,   'P1',  'P2',  'P3',  'P4',  'P5',  'P6'],
                       ['M1',    3  ,   3  ,   2  ,   5  ,   2  ,  1   ],
                       ['M2',    4  ,   1  ,   1  ,   2  ,   2  ,  1   ],
                       ['M3',    2  ,   2  ,   5  ,   1  ,   1  ,  2   ]]))

time = to_df(np.array([[ '' ,  'P1',  'P2',  'P3',  'P4',  'P5',  'P6'],
                       ['M1',   1.3 ,  1.3 ,  1.2 ,  1.5 ,  1.2 ,  1.1],
                       ['M2',   1.4 ,  1.1 ,  1.1 ,  1.2 ,  1.2 ,  1.1],
                       ['M3',   1.2 ,  1.2 ,  1.5 ,  1.1 ,  1.1 ,  1.2]]))

In [ ]:
def manufacture(capacity, required, cost, time):
  ## ADD CODE FOR PART (c) BELOW ##
  

In [ ]:
# When you complete part (c), run the below command
manufacture(capacity_c, required_c, cost_c, time)

**(d)** Create a new function called `manufacture_d` that solves the preceding problem again but with the objective function changed to *minimize total
machine-hours* rather than total cost.

Hint: Copy your code from part (c) and modify it to use this new machine-hours based objective.

In [ ]:
def manufacture_d(capacity, required, cost, time):
  ## ADD CODE FOR PART (d) BELOW ##
  

In [ ]:
# When you complete part (d), run the below command
manufacture_d(capacity_c, required_c, cost_c, time) 